In [39]:
import requests
from datetime import datetime, timedelta
import pandas as pd
import numpy as np
import time
import json
import os

try:
    import yfinance as yf
    YFINANCE_AVAILABLE = True
except ImportError:
    YFINANCE_AVAILABLE = False
    print("⚠️  yfinance not installed - IV data will not be available")
    print("   Install with: pip install yfinance")

# ============================================================================
# CONFIGURATION
# ============================================================================

ALPHAVANTAGE_KEYS = [
    "HPCFVLGHWHQU0QTY",
    "VL7Z4WRK8T5MJPK5",
    "DYU6F4AG3IL03321",
    "EXMUX4OSACRK51NZ"
]
CURRENT_KEY_INDEX = 0
RATE_LIMITED_KEYS = set()
CACHE_FILE = "earnings_cache.json"
RATE_LIMIT_FILE = "rate_limits.json"

# ============================================================================
# CACHING & RATE LIMITS
# ============================================================================

def load_cache():
    """Load cached earnings data"""
    if os.path.exists(CACHE_FILE):
        with open(CACHE_FILE, 'r') as f:
            return json.load(f)
    return {}

def save_cache(cache):
    """Save earnings data to cache"""
    with open(CACHE_FILE, 'w') as f:
        json.dump(cache, f, indent=2, default=str)

def load_rate_limits():
    """Load rate limit state with 24-hour reset"""
    if os.path.exists(RATE_LIMIT_FILE):
        try:
            with open(RATE_LIMIT_FILE, 'r') as f:
                data = json.load(f)
            
            now = datetime.now().timestamp()
            active_limits = {}
            
            for key_idx, info in data.items():
                reset_time = info.get('reset_time', 0)
                if reset_time > now:
                    active_limits[int(key_idx)] = info
            
            return set(active_limits.keys())
        except:
            return set()
    return set()

def save_rate_limits(rate_limited_keys):
    """Persist rate limit state with 24-hour expiry"""
    reset_time = (datetime.now() + timedelta(hours=24)).timestamp()
    
    data = {
        str(k): {
            'reset_time': reset_time,
            'limited_at': datetime.now().isoformat()
        } 
        for k in rate_limited_keys
    }
    
    with open(RATE_LIMIT_FILE, 'w') as f:
        json.dump(data, f, indent=2)

# ============================================================================
# IMPLIED VOLATILITY
# ============================================================================

def get_current_iv(ticker, dte_target=45, retry_count=2):
    """
    Fetch current implied volatility from Yahoo Finance options chain
    Returns ATM IV for expiration closest to target DTE
    """
    if not YFINANCE_AVAILABLE:
        return None
    
    try:
        stock = yf.Ticker(ticker)
        
        # Get current price
        hist = stock.history(period='1d')
        if hist.empty:
            return None
        current_price = hist['Close'].iloc[-1]
        
        # Get available expirations
        expirations = stock.options
        if not expirations:
            return None
        
        # Find expiration closest to target DTE
        today = datetime.now()
        target_exp = None
        min_diff = 999
        
        for exp_str in expirations:
            exp_date = datetime.strptime(exp_str, '%Y-%m-%d')
            dte = (exp_date - today).days
            if abs(dte - dte_target) < min_diff:
                min_diff = abs(dte - dte_target)
                target_exp = exp_str
        
        if not target_exp:
            return None
        
        # Get options chain
        chain = stock.option_chain(target_exp)
        calls = chain.calls
        
        if calls.empty or 'impliedVolatility' not in calls.columns:
            return None
        
        # Find ATM strike
        calls['strike_diff'] = abs(calls['strike'] - current_price)
        atm_idx = calls['strike_diff'].idxmin()
        atm_call = calls.loc[atm_idx]
        
        iv_pct = atm_call['impliedVolatility'] * 100
        actual_dte = (datetime.strptime(target_exp, '%Y-%m-%d') - today).days
        
        return {
            'iv': round(iv_pct, 1),
            'dte': actual_dte,
            'strike': atm_call['strike'],
            'expiration': target_exp
        }
    
    except Exception as e:
        if retry_count > 0:
            time.sleep(1)
            return get_current_iv(ticker, dte_target, retry_count - 1)
        return None

# ============================================================================
# EARNINGS DATA
# ============================================================================

def get_earnings_details(ticker, use_cache=True, debug=False):
    """Get earnings announcement dates from Alpha Vantage with automatic key rotation"""
    global CURRENT_KEY_INDEX, RATE_LIMITED_KEYS
    
    # Check cache first
    cache = load_cache()
    if use_cache and ticker in cache:
        return [
            {'date': datetime.fromisoformat(e['date']), 'time': e['time']} 
            for e in cache[ticker]
        ], "cached"
    
    # Load persisted rate limits
    if not RATE_LIMITED_KEYS:
        RATE_LIMITED_KEYS = load_rate_limits()
    
    # Check if all keys exhausted
    if len(RATE_LIMITED_KEYS) >= len(ALPHAVANTAGE_KEYS):
        if debug:
            print(f"  {ticker}: All API keys exhausted")
        return [], "rate_limited_all"
    
    # Try available keys
    max_attempts = len(ALPHAVANTAGE_KEYS) - len(RATE_LIMITED_KEYS)
    for attempt in range(max_attempts):
        # Skip rate-limited keys
        while CURRENT_KEY_INDEX in RATE_LIMITED_KEYS:
            CURRENT_KEY_INDEX = (CURRENT_KEY_INDEX + 1) % len(ALPHAVANTAGE_KEYS)
        
        current_key = ALPHAVANTAGE_KEYS[CURRENT_KEY_INDEX]
        url = f"https://www.alphavantage.co/query?function=EARNINGS&symbol={ticker}&apikey={current_key}"
        
        try:
            response = requests.get(url, timeout=10)
            data = response.json()
            
            # Handle API errors
            error_msg = data.get('Note', data.get('Information', ''))
            if error_msg:
                # Check for rate limit
                if any(phrase in error_msg.lower() for phrase in ['rate limit', 'call frequency']):
                    RATE_LIMITED_KEYS.add(CURRENT_KEY_INDEX)
                    save_rate_limits(RATE_LIMITED_KEYS)
                    
                    CURRENT_KEY_INDEX = (CURRENT_KEY_INDEX + 1) % len(ALPHAVANTAGE_KEYS)
                    
                    if len(RATE_LIMITED_KEYS) >= len(ALPHAVANTAGE_KEYS):
                        return [], "rate_limited_all"
                    
                    time.sleep(1)
                    continue
                
                return [], "api_error"
            
            # Parse earnings data
            if 'quarterlyEarnings' not in data:
                return [], "no_earnings"
            
            earnings_info = []
            for quarter in data['quarterlyEarnings']:
                reported_date = quarter.get('reportedDate')
                reported_time = quarter.get('reportTime') or quarter.get('reportedTime', 'amc')
                
                if reported_date:
                    # Normalize time values
                    time_map = {'pre-market': 'bmo', 'post-market': 'amc'}
                    normalized_time = time_map.get(reported_time.lower(), reported_time.lower())
                    
                    earnings_info.append({
                        'date': datetime.strptime(reported_date, '%Y-%m-%d'),
                        'time': normalized_time
                    })
            
            # Cache the results
            cache[ticker] = [
                {'date': e['date'].isoformat(), 'time': e['time']} 
                for e in earnings_info
            ]
            save_cache(cache)
            
            return sorted(earnings_info, key=lambda x: x['date'], reverse=True), "success"
        
        except Exception:
            if attempt < max_attempts - 1:
                CURRENT_KEY_INDEX = (CURRENT_KEY_INDEX + 1) % len(ALPHAVANTAGE_KEYS)
                continue
            return [], "exception"
    
    return [], "unknown_error"

# ============================================================================
# PRICE DATA
# ============================================================================

def get_yahoo_price_data(ticker, start_date, end_date):
    """Get historical closing prices from Yahoo Finance"""
    start_ts = int(start_date.timestamp())
    end_ts = int(end_date.timestamp())
    
    url = f"https://query1.finance.yahoo.com/v8/finance/chart/{ticker}"
    params = {'period1': start_ts, 'period2': end_ts, 'interval': '1d'}
    headers = {'User-Agent': 'Mozilla/5.0'}
    
    try:
        response = requests.get(url, params=params, headers=headers, timeout=10)
        data = response.json()
        
        result = data['chart']['result'][0]
        timestamps = result['timestamp']
        closes = result['indicators']['quote'][0]['close']
        
        df = pd.DataFrame({
            'date': [datetime.fromtimestamp(ts) for ts in timestamps],
            'close': closes
        })
        df.set_index('date', inplace=True)
        df.dropna(inplace=True)
        
        return df
    
    except Exception as e:
        print(f"❌ Error fetching prices for {ticker}: {e}")
        return pd.DataFrame()

# ============================================================================
# CALCULATIONS
# ============================================================================

def find_nearest_price(price_data, target_date):
    """Find closing price on nearest trading day"""
    if price_data.empty:
        return None, None
    
    start = target_date - timedelta(days=7)
    end = target_date + timedelta(days=7)
    nearby = price_data[(price_data.index >= start) & (price_data.index <= end)]
    
    if nearby.empty:
        return None, None
    
    time_diffs = (nearby.index - target_date).to_series().abs()
    closest_idx = time_diffs.argmin()
    return nearby.iloc[closest_idx]['close'], nearby.index[closest_idx]

def get_reference_price(price_data, earnings_date, timing):
    """Get entry price based on earnings timing"""
    target_date = earnings_date - timedelta(days=1) if timing == 'bmo' else earnings_date
    return find_nearest_price(price_data, target_date)

def calculate_historical_volatility(price_data, earnings_date, lookback_days=30):
    """Calculate annualized historical volatility"""
    end_date = earnings_date - timedelta(days=1)
    start_date = end_date - timedelta(days=lookback_days + 10)
    
    window = price_data[(price_data.index >= start_date) & (price_data.index <= end_date)]
    
    if len(window) < 20:
        return None
    
    returns = window['close'].pct_change().dropna()
    daily_vol = returns.std()
    annual_vol = daily_vol * np.sqrt(252)
    
    return annual_vol

def get_volatility_tier(hvol):
    """Map historical volatility to strike width multiplier"""
    hvol_pct = hvol * 100
    
    if hvol_pct < 25:
        return 1.0
    elif hvol_pct < 35:
        return 1.2
    elif hvol_pct < 45:
        return 1.4
    else:
        return 1.5

def calculate_stats(data):
    """Calculate containment and directional statistics"""
    total = len(data)
    moves = np.array([d['move'] for d in data])
    widths = np.array([d['width'] for d in data])
    
    # Containment
    stays_within = sum(1 for i, m in enumerate(moves) if abs(m) <= widths[i])
    breaks_up = sum(1 for i, m in enumerate(moves) if m > widths[i])
    breaks_down = sum(1 for i, m in enumerate(moves) if m < -widths[i])
    
    # Overall directional bias
    up_moves = sum(1 for m in moves if m > 0)
    overall_bias = (up_moves / total) * 100
    
    # Break directional bias
    total_breaks = breaks_up + breaks_down
    if total_breaks > 0:
        break_bias = (breaks_up / total_breaks) * 100
    else:
        break_bias = 50.0
    
    # Magnitude-weighted average move
    avg_move = np.mean(moves)
    avg_move_vs_width = (avg_move / np.mean(widths)) * 100 if np.mean(widths) > 0 else 0
    
    return {
        'total': total,
        'containment': (stays_within / total) * 100,
        'breaks_up': breaks_up,
        'breaks_down': breaks_down,
        'overall_bias': overall_bias,
        'break_bias': break_bias,
        'avg_move_pct': avg_move,
        'drift_vs_width': avg_move_vs_width,
        'avg_width': np.mean(widths)
    }

# ============================================================================
# STRATEGY LOGIC
# ============================================================================

def determine_strategy(stats_45, stats_90):
    """
    Determine trading strategy based on containment and directional patterns
    Returns: (strategy_name, reason_string)
    """
    rec_parts = []
    bias_reasons = []
    
    # CATEGORY 1: CONTAINMENT STRATEGIES (IC)
    if stats_90['containment'] >= 69.5:
        break_ratio = max(stats_90['breaks_up'], stats_90['breaks_down']) / (
            min(stats_90['breaks_up'], stats_90['breaks_down']) + 1
        )
        
        if break_ratio < 2:
            rec_parts.append("IC90")
        elif stats_90['break_bias'] >= 70:
            rec_parts.append("IC90⚠↑")
        else:
            rec_parts.append("IC90⚠↓")
            
    elif stats_45['containment'] >= 69.5:
        break_ratio = max(stats_45['breaks_up'], stats_45['breaks_down']) / (
            min(stats_45['breaks_up'], stats_45['breaks_down']) + 1
        )
        
        if break_ratio < 2:
            rec_parts.append("IC45")
        elif stats_45['break_bias'] >= 70:
            rec_parts.append("IC45⚠↑")
        else:
            rec_parts.append("IC45⚠↓")
    
    # CATEGORY 2: DIRECTIONAL BIAS
    else:
        has_upward_edge = False
        has_downward_edge = False
        
        # Check for UPWARD edge
        if stats_90['overall_bias'] >= 65:
            has_upward_edge = True
            bias_reasons.append(f"{stats_90['overall_bias']:.0f}% bias")
        
        if stats_90['breaks_up'] >= stats_90['breaks_down'] * 1.5 and stats_90['breaks_up'] >= 2:
            has_upward_edge = True
            bias_reasons.append(f"{stats_90['breaks_up']}:{stats_90['breaks_down']}↑ breaks")
        
        if stats_90['avg_move_pct'] >= 3.0:
            has_upward_edge = True
            bias_reasons.append(f"{stats_90['avg_move_pct']:+.1f}% drift")
        
        # Check for DOWNWARD edge
        if stats_90['overall_bias'] <= 35:
            has_downward_edge = True
            bias_reasons.append(f"{stats_90['overall_bias']:.0f}% bias")
        
        if stats_90['breaks_down'] >= stats_90['breaks_up'] * 1.5 and stats_90['breaks_down'] >= 2:
            has_downward_edge = True
            bias_reasons.append(f"{stats_90['breaks_up']}:{stats_90['breaks_down']}↓ breaks")
        
        if stats_90['avg_move_pct'] <= -3.0:
            has_downward_edge = True
            bias_reasons.append(f"{stats_90['avg_move_pct']:+.1f}% drift")
        
        # Assign directional recommendation
        if has_upward_edge:
            reason_str = ", ".join(bias_reasons)
            rec_parts.append(f"BIAS↑ ({reason_str})")
        elif has_downward_edge:
            reason_str = ", ".join(bias_reasons)
            rec_parts.append(f"BIAS↓ ({reason_str})")
    
    return rec_parts[0] if rec_parts else "SKIP"

# ============================================================================
# MAIN ANALYSIS
# ============================================================================

def analyze_earnings_movement(ticker, lookback_quarters=24, verbose=True, debug=False):
    """Analyze post-earnings movements with volatility-adjusted strikes"""
    
    if verbose:
        print(f"\n{'='*75}")
        print(f"📊 {ticker} - Post-Earnings Containment Analysis")
        print(f"{'='*75}")
    
    # Get earnings dates
    earnings_info, status = get_earnings_details(ticker, debug=debug)
    if not earnings_info:
        return None, status
    
    # Filter to past earnings
    today = datetime.now()
    past_earnings = [e for e in earnings_info if e['date'] < today][:lookback_quarters]
    
    if len(past_earnings) < 10:
        if verbose:
            print(f"⚠️  Insufficient data: only {len(past_earnings)} earnings periods")
        return None, "insufficient_quarters"
    
    # Get price data
    oldest = min([e['date'] for e in past_earnings]) - timedelta(days=120)
    price_data = get_yahoo_price_data(ticker, oldest, today)
    
    if price_data.empty:
        return None, "no_price_data"
    
    # Collect movement data
    data_45 = []
    data_90 = []
    hvol_list = []
    
    for earnings in past_earnings:
        hvol = calculate_historical_volatility(price_data, earnings['date'])
        if hvol is None:
            continue
        
        hvol_list.append(hvol * 100)
        strike_std = get_volatility_tier(hvol)
        
        # Get entry price
        ref_price, ref_date = get_reference_price(price_data, earnings['date'], earnings['time'])
        if ref_price is None:
            continue
        
        # Calculate strike widths
        dte_45_factor = np.sqrt(45 / 365)
        dte_90_factor = np.sqrt(90 / 365)
        strike_width_45 = hvol * dte_45_factor * strike_std * 100
        strike_width_90 = hvol * dte_90_factor * strike_std * 100
        
        # Test 45-day outcome
        target_45 = earnings['date'] + timedelta(days=45)
        if target_45 <= today:
            price_45, date_45 = find_nearest_price(price_data, target_45)
            if price_45 is not None:
                move_45 = (price_45 - ref_price) / ref_price * 100
                data_45.append({
                    'move': move_45,
                    'width': strike_width_45,
                    'hvol': hvol * 100,
                    'date': earnings['date'].strftime('%Y-%m-%d')
                })
        
        # Test 90-day outcome
        target_90 = earnings['date'] + timedelta(days=90)
        if target_90 <= today:
            price_90, date_90 = find_nearest_price(price_data, target_90)
            if price_90 is not None:
                move_90 = (price_90 - ref_price) / ref_price * 100
                data_90.append({
                    'move': move_90,
                    'width': strike_width_90,
                    'hvol': hvol * 100,
                    'date': earnings['date'].strftime('%Y-%m-%d')
                })
    
    if len(data_45) < 10 or len(data_90) < 10:
        if verbose:
            print(f"⚠️  Insufficient valid data")
        return None, "insufficient_valid_data"
    
    # Calculate statistics
    stats_45 = calculate_stats(data_45)
    stats_90 = calculate_stats(data_90)
    avg_hvol = np.mean(hvol_list)
    avg_tier = get_volatility_tier(avg_hvol / 100)
    
    # Determine strategy
    recommendation = determine_strategy(stats_45, stats_90)
    
    if verbose:
        print(f"\n📊 {ticker} | {avg_hvol:.1f}% HVol | {avg_tier:.1f} std (±{stats_90['avg_width']:.1f}%)")
        print(f"\n  45-Day: {stats_45['total']}/{lookback_quarters} tested")
        print(f"    Containment: {stats_45['containment']:.0f}%")
        print(f"    Breaks: Up {stats_45['breaks_up']}, Down {stats_45['breaks_down']}")
        print(f"    Overall Bias: {stats_45['overall_bias']:.0f}% up")
        print(f"    Break Bias: {stats_45['break_bias']:.0f}% of breaks were upward")
        print(f"    Avg Drift: {stats_45['avg_move_pct']:+.1f}% ({stats_45['drift_vs_width']:+.0f}% of width)")
        
        print(f"\n  90-Day: {stats_90['total']}/{lookback_quarters} tested")
        print(f"    Containment: {stats_90['containment']:.0f}%")
        print(f"    Breaks: Up {stats_90['breaks_up']}, Down {stats_90['breaks_down']}")
        print(f"    Overall Bias: {stats_90['overall_bias']:.0f}% up")
        print(f"    Break Bias: {stats_90['break_bias']:.0f}% of breaks were upward")
        print(f"    Avg Drift: {stats_90['avg_move_pct']:+.1f}% ({stats_90['drift_vs_width']:+.0f}% of width)")
        
        print(f"\n  💡 Strategy: {recommendation}")
    
    summary = {
        'ticker': ticker,
        'hvol': round(avg_hvol, 1),
        'tier': round(avg_tier, 1),
        'strike_width': round(stats_90['avg_width'], 1),
        '45d_contain': round(stats_45['containment'], 0),
        '45d_breaks_up': stats_45['breaks_up'],
        '45d_breaks_dn': stats_45['breaks_down'],
        '45d_overall_bias': round(stats_45['overall_bias'], 0),
        '45d_break_bias': round(stats_45['break_bias'], 0),
        '45d_drift': round(stats_45['avg_move_pct'], 1),
        '90d_contain': round(stats_90['containment'], 0),
        '90d_breaks_up': stats_90['breaks_up'],
        '90d_breaks_dn': stats_90['breaks_down'],
        '90d_overall_bias': round(stats_90['overall_bias'], 0),
        '90d_break_bias': round(stats_90['break_bias'], 0),
        '90d_drift': round(stats_90['avg_move_pct'], 1),
        'strategy': recommendation
    }
    
    return summary, "success"

# ============================================================================
# BATCH PROCESSING
# ============================================================================

def format_break_ratio(up_breaks, down_breaks, break_bias):
    """Format break ratio with directional arrow if 2:1 edge exists"""
    if up_breaks == 0 and down_breaks == 0:
        return "0:0"
    
    if break_bias >= 66.7:
        return f"{up_breaks}:{down_breaks}↑"
    elif break_bias <= 33.3:
        return f"{up_breaks}:{down_breaks}↓"
    else:
        return f"{up_breaks}:{down_breaks}"

def batch_analyze(tickers, lookback_quarters=24, debug=False, fetch_iv=True):
    """Analyze multiple tickers with progress tracking"""
    
    print("\n" + "="*75)
    print(f"EARNINGS CONTAINMENT ANALYZER - v2.3")
    print(f"Lookback: {lookback_quarters} quarters (~{lookback_quarters/4:.0f} years)")
    if fetch_iv and YFINANCE_AVAILABLE:
        print(f"Current IV from Yahoo Finance (15-20min delayed)")
    print("="*75)
    
    # Show rate limit status
    rate_limited_keys = load_rate_limits()
    if rate_limited_keys:
        available = len(ALPHAVANTAGE_KEYS) - len(rate_limited_keys)
        print(f"\n⚠️  Rate Limit: {available}/{len(ALPHAVANTAGE_KEYS)} API keys available")
    else:
        print(f"\n✓ All {len(ALPHAVANTAGE_KEYS)} API keys available")
    
    results = []
    fetch_summary = {'cached': [], 'api': [], 'failed': []}
    iv_summary = {'success': [], 'failed': []}
    
    for i, ticker in enumerate(tickers, 1):
        print(f"\r[{i}/{len(tickers)}] Processing {ticker}...", end='', flush=True)
        
        cache = load_cache()
        from_cache = ticker in cache
        
        summary, status = analyze_earnings_movement(ticker, lookback_quarters, verbose=False, debug=debug)
        
        if summary:
            # Fetch current IV
            if fetch_iv and YFINANCE_AVAILABLE:
                iv_data = get_current_iv(ticker, dte_target=45)
                if iv_data:
                    summary['current_iv'] = iv_data['iv']
                    summary['iv_dte'] = iv_data['dte']
                    iv_premium = ((iv_data['iv'] - summary['hvol']) / summary['hvol']) * 100
                    summary['iv_premium'] = round(iv_premium, 1)
                    iv_summary['success'].append(ticker)
                else:
                    summary['current_iv'] = None
                    summary['iv_dte'] = None
                    summary['iv_premium'] = None
                    iv_summary['failed'].append(ticker)
            else:
                summary['current_iv'] = None
                summary['iv_dte'] = None
                summary['iv_premium'] = None
            
            results.append(summary)
            if from_cache:
                fetch_summary['cached'].append(ticker)
            else:
                fetch_summary['api'].append(ticker)
        else:
            fetch_summary['failed'].append(ticker)
        
        time.sleep(0.5)
    
    print("\r" + " " * 80 + "\r", end='')
    
    # Print fetch summary
    print(f"\n📊 FETCH SUMMARY")
    print(f"{'='*75}")
    if fetch_summary['cached']:
        print(f"✓ Earnings Cached ({len(fetch_summary['cached'])}): {', '.join(fetch_summary['cached'][:5])}{'...' if len(fetch_summary['cached']) > 5 else ''}")
    if fetch_summary['api']:
        print(f"✓ Earnings API ({len(fetch_summary['api'])}): {', '.join(fetch_summary['api'])}")
    if fetch_iv and YFINANCE_AVAILABLE:
        if iv_summary['success']:
            print(f"✓ IV Retrieved ({len(iv_summary['success'])}): {', '.join(iv_summary['success'][:5])}{'...' if len(iv_summary['success']) > 5 else ''}")
        if iv_summary['failed']:
            print(f"✗ IV Failed ({len(iv_summary['failed'])}): {', '.join(iv_summary['failed'])}")
    if fetch_summary['failed']:
        print(f"✗ Analysis Failed ({len(fetch_summary['failed'])}): {', '.join(fetch_summary['failed'])}")
    
    if not results:
        print("\n⚠️  No valid results")
        return None
    
    df = pd.DataFrame(results)
    
    # Calculate 45-day strike width
    df['45d_width'] = df.apply(lambda x: round(x['strike_width'] * np.sqrt(45/90), 1), axis=1)
    
    # Format break ratios
    df['45_break_fmt'] = df.apply(
        lambda x: format_break_ratio(x['45d_breaks_up'], x['45d_breaks_dn'], x['45d_break_bias']), 
        axis=1
    )
    df['90_break_fmt'] = df.apply(
        lambda x: format_break_ratio(x['90d_breaks_up'], x['90d_breaks_dn'], x['90d_break_bias']), 
        axis=1
    )
    
    # Format strategies
    df['strategy_display'] = df['strategy'].apply(lambda x: 
        x.replace('BIAS↑', 'BIAS↑').replace('BIAS↓', 'BIAS↓') if 'BIAS' in x else x
    )
    
    # Display results
    print(f"\n{'='*110}")
    print("BACKTEST RESULTS")
    print("="*110)
    
    # Prepare display columns
    display_cols = {
        'Ticker': df['ticker'],
        'HVol%': df['hvol'].astype(int),
    }
    
    # Add IV columns if available
    if 'current_iv' in df.columns and df['current_iv'].notna().any():
        display_cols['CurIV%'] = df['current_iv'].apply(lambda x: f"{int(x)}" if pd.notna(x) else "N/A")
        display_cols['IVPrem'] = df['iv_premium'].apply(lambda x: f"{x:+.0f}%" if pd.notna(x) else "N/A")
        display_cols['|'] = '|'
    
    display_cols.update({
        '90D%': df['90d_contain'].astype(int),
        '90Bias': df['90d_overall_bias'].astype(int),
        '90Break': df['90_break_fmt'],
        '90Drift': df['90d_drift'].apply(lambda x: f"{x:+.1f}%"),
        ' | ': '|',
        'Pattern': df['strategy_display']
    })
    
    display_df = pd.DataFrame(display_cols)
    
    print(display_df.to_string(index=False))
    
    # Summary insights
    print(f"\n{'='*110}")
    print("KEY TAKEAWAYS:")
    print("="*110)
    
    # Pattern counts
    ic_count = len(df[df['strategy'].str.contains('IC', na=False)])
    bias_up_count = len(df[df['strategy'].str.contains('BIAS↑', na=False)])
    bias_down_count = len(df[df['strategy'].str.contains('BIAS↓', na=False)])
    skip_count = len(df[df['strategy'] == 'SKIP'])
    
    print(f"\n📊 Pattern Summary: {ic_count} IC candidates | {bias_up_count} Upward bias | {bias_down_count} Downward bias | {skip_count} No edge")
    
    # IV Context (if available)
    if 'iv_premium' in df.columns and df['iv_premium'].notna().any():
        elevated = df[df['iv_premium'] >= 15].sort_values('iv_premium', ascending=False)
        depressed = df[df['iv_premium'] <= -15].sort_values('iv_premium')
        
        print(f"\n💰 IV Landscape:")
        if not elevated.empty:
            tickers_str = ', '.join([f"{row['ticker']}(+{row['iv_premium']:.0f}%)" for _, row in elevated.head(5).iterrows()])
            print(f"  Rich Premium (≥15%): {tickers_str}")
        if not depressed.empty:
            tickers_str = ', '.join([f"{row['ticker']}({row['iv_premium']:.0f}%)" for _, row in depressed.head(3).iterrows()])
            print(f"  Thin Premium (≤-15%): {tickers_str}")
        
        normal_count = len(df[(df['iv_premium'] > -15) & (df['iv_premium'] < 15)])
        if normal_count > 0:
            print(f"  Normal Range: {normal_count} tickers")
    
    # Asymmetric ICs
    ic_up_skew = df[(df['strategy'].str.contains('IC.*⚠↑', regex=True, na=False))]
    ic_down_skew = df[(df['strategy'].str.contains('IC.*⚠↓', regex=True, na=False))]
    
    if not ic_up_skew.empty or not ic_down_skew.empty:
        print(f"\n⚠️  Asymmetric ICs:")
        if not ic_up_skew.empty:
            print(f"  Upside risk: {', '.join(ic_up_skew['ticker'].tolist())}")
        if not ic_down_skew.empty:
            print(f"  Downside risk: {', '.join(ic_down_skew['ticker'].tolist())}")
    
    # Strong directional edges
    strong_bias = df[
        (df['strategy'].str.contains('BIAS', na=False)) &
        ((df['90d_overall_bias'] >= 70) | (df['90d_overall_bias'] <= 30))
    ]
    if not strong_bias.empty:
        print(f"\n📈 Strong Directional Signals:")
        for _, row in strong_bias.iterrows():
            direction = "↑" if row['90d_overall_bias'] >= 70 else "↓"
            print(f"  {row['ticker']}: {row['90d_overall_bias']:.0f}% bias {direction}, {row['90_break_fmt']} breaks, {row['90d_drift']:+.1f}% drift")
    
    print(f"\n💡 Remember: Past patterns ≠ Future results. IV context shows current opportunity cost.")
    
    return df

# ============================================================================
# RUN
# ============================================================================

if __name__ == "__main__":
    # Test tickers
    tickers = ["DAL", "PEP", "FAST", "BLK", "C", "DPZ", "GS", "JNJ", "JPM", "WFC", 
               "OMC", "ABT", "BAC", "CFG", "MS", "PGR", "PLD", "PNC", "SYF", "JBHT", 
               "UAL", "BK", "KEY", "MMC", "MTB", "SCHW", "SNA", "TRV", "USB", "CSX", 
               "AXP", "FITB", "HBAN", "RF", "SLB", "STT", "TFC", "STLD"]
    
    results = batch_analyze(tickers, lookback_quarters=24, debug=False)


EARNINGS CONTAINMENT ANALYZER - v2.3
Lookback: 24 quarters (~6 years)
Current IV from Yahoo Finance (15-20min delayed)

✓ All 4 API keys available
                                                                                
📊 FETCH SUMMARY
✓ Earnings Cached (38): DAL, PEP, FAST, BLK, C...
✓ IV Retrieved (38): DAL, PEP, FAST, BLK, C...

BACKTEST RESULTS
Ticker  HVol% CurIV% IVPrem |  90D%  90Bias 90Break 90Drift  |                                     Pattern
   DAL     43     49   +14% |    67      62     5:3   +3.5%   |                                       IC45
   PEP     18     26   +44% |    74      65     3:3   +1.3%   |                                       IC90
  FAST     24     31   +26% |    78      61     3:2   +4.0%   |                                       IC90
   BLK     28     29    +4% |    61      57     5:4   +4.9%   |                        BIAS↑ (+4.9% drift)
     C     34     31    -8% |    70      48     4:3   +2.9%   |                                       IC9

In [38]:
import requests
from datetime import datetime, timedelta
import pandas as pd
import numpy as np
import time
import json
import os

try:
    import yfinance as yf
    YFINANCE_AVAILABLE = True
except ImportError:
    YFINANCE_AVAILABLE = False
    print("⚠️  yfinance not installed - IV data will not be available")
    print("   Install with: pip install yfinance")

# API Configuration
ALPHAVANTAGE_KEYS = [
    "HPCFVLGHWHQU0QTY",
    "VL7Z4WRK8T5MJPK5",
    "DYU6F4AG3IL03321",
    "EXMUX4OSACRK51NZ"
]
CURRENT_KEY_INDEX = 0
RATE_LIMITED_KEYS = set()
CACHE_FILE = "earnings_cache.json"
RATE_LIMIT_FILE = "rate_limits.json"  # NEW v2.1: Persistent rate limit tracking

# ============================================================================
# CACHING
# ============================================================================

def load_cache():
    """Load cached earnings data"""
    if os.path.exists(CACHE_FILE):
        with open(CACHE_FILE, 'r') as f:
            return json.load(f)
    return {}

def save_cache(cache):
    """Save earnings data to cache"""
    with open(CACHE_FILE, 'w') as f:
        json.dump(cache, f, indent=2, default=str)

# ============================================================================
# RATE LIMIT PERSISTENCE - NEW v2.1
# ============================================================================

def load_rate_limits():
    """
    Load rate limit state with timestamps
    Alpha Vantage: 25 calls per day, resets daily
    """
    if os.path.exists(RATE_LIMIT_FILE):
        try:
            with open(RATE_LIMIT_FILE, 'r') as f:
                data = json.load(f)
            
            # Reset limits older than 24 hours
            now = datetime.now().timestamp()
            active_limits = {}
            
            for key_idx, info in data.items():
                reset_time = info.get('reset_time', 0)
                if reset_time > now:
                    active_limits[int(key_idx)] = info
            
            return set(active_limits.keys())
        except:
            return set()
    return set()

def save_rate_limits(rate_limited_keys):
    """
    Persist rate limit state with 24-hour expiry
    """
    reset_time = (datetime.now() + timedelta(hours=24)).timestamp()
    
    data = {
        str(k): {
            'reset_time': reset_time,
            'limited_at': datetime.now().isoformat()
        } 
        for k in rate_limited_keys
    }
    
    with open(RATE_LIMIT_FILE, 'w') as f:
        json.dump(data, f, indent=2)

# ============================================================================
# CURRENT IMPLIED VOLATILITY - NEW v2.2
# ============================================================================

def get_current_iv(ticker, dte_target=45, retry_count=2):
    """
    Fetch current implied volatility from Yahoo Finance options chain
    
    Returns ATM IV for expiration closest to target DTE
    Data is delayed 15-20 minutes (free tier)
    
    Returns None if:
    - yfinance not installed
    - No options available
    - API error
    """
    if not YFINANCE_AVAILABLE:
        return None
    
    try:
        stock = yf.Ticker(ticker)
        
        # Get current price
        hist = stock.history(period='1d')
        if hist.empty:
            return None
        current_price = hist['Close'].iloc[-1]
        
        # Get available expirations
        expirations = stock.options
        if not expirations:
            return None
        
        # Find expiration closest to target DTE
        today = datetime.now()
        target_exp = None
        min_diff = 999
        
        for exp_str in expirations:
            exp_date = datetime.strptime(exp_str, '%Y-%m-%d')
            dte = (exp_date - today).days
            if abs(dte - dte_target) < min_diff:
                min_diff = abs(dte - dte_target)
                target_exp = exp_str
        
        if not target_exp:
            return None
        
        # Get options chain
        chain = stock.option_chain(target_exp)
        calls = chain.calls
        
        if calls.empty or 'impliedVolatility' not in calls.columns:
            return None
        
        # Find ATM strike (closest to current price)
        calls['strike_diff'] = abs(calls['strike'] - current_price)
        atm_idx = calls['strike_diff'].idxmin()
        atm_call = calls.loc[atm_idx]
        
        iv_decimal = atm_call['impliedVolatility']
        iv_pct = iv_decimal * 100
        
        actual_dte = (datetime.strptime(target_exp, '%Y-%m-%d') - today).days
        
        return {
            'iv': round(iv_pct, 1),
            'dte': actual_dte,
            'strike': atm_call['strike'],
            'expiration': target_exp
        }
    
    except Exception as e:
        if retry_count > 0:
            time.sleep(1)
            return get_current_iv(ticker, dte_target, retry_count - 1)
        return None

# ============================================================================
# ALPHA VANTAGE - EARNINGS DATA
# ============================================================================

def get_earnings_details(ticker, use_cache=True, debug=False):
    """Get earnings announcement dates from Alpha Vantage with automatic key rotation"""
    global CURRENT_KEY_INDEX, RATE_LIMITED_KEYS
    
    # Check cache first
    cache = load_cache()
    if use_cache and ticker in cache:
        return [
            {'date': datetime.fromisoformat(e['date']), 'time': e['time']} 
            for e in cache[ticker]
        ], "cached"
    
    # NEW v2.1: Load persisted rate limits at start
    if not RATE_LIMITED_KEYS:  # Only load once per run
        RATE_LIMITED_KEYS = load_rate_limits()
        if RATE_LIMITED_KEYS and debug:
            print(f"  Loaded {len(RATE_LIMITED_KEYS)} rate-limited keys from previous run")
    
    # Check if all keys exhausted
    if len(RATE_LIMITED_KEYS) >= len(ALPHAVANTAGE_KEYS):
        if debug:
            print(f"  {ticker}: All API keys exhausted")
        return [], "rate_limited_all"
    
    # Try available keys
    max_attempts = len(ALPHAVANTAGE_KEYS) - len(RATE_LIMITED_KEYS)
    for attempt in range(max_attempts):
        # Skip rate-limited keys
        while CURRENT_KEY_INDEX in RATE_LIMITED_KEYS:
            CURRENT_KEY_INDEX = (CURRENT_KEY_INDEX + 1) % len(ALPHAVANTAGE_KEYS)
        
        current_key = ALPHAVANTAGE_KEYS[CURRENT_KEY_INDEX]
        key_suffix = current_key[-4:]  # Last 4 chars for identification
        url = f"https://www.alphavantage.co/query?function=EARNINGS&symbol={ticker}&apikey={current_key}"
        
        if debug:
            print(f"  {ticker}: Trying key ...{key_suffix} (attempt {attempt + 1}/{max_attempts})")
        
        try:
            response = requests.get(url, timeout=10)
            data = response.json()
            
            # Handle API errors
            error_msg = data.get('Note', data.get('Information', ''))
            if error_msg:
                if debug:
                    print(f"  {ticker}: Key ...{key_suffix} returned: {error_msg[:80]}")
                
                # Check for rate limit
                if any(phrase in error_msg.lower() for phrase in ['rate limit', 'call frequency']):
                    RATE_LIMITED_KEYS.add(CURRENT_KEY_INDEX)
                    save_rate_limits(RATE_LIMITED_KEYS)  # NEW v2.1: Persist immediately
                    
                    if debug:
                        print(f"  {ticker}: Key ...{key_suffix} marked as rate-limited ({len(RATE_LIMITED_KEYS)}/{len(ALPHAVANTAGE_KEYS)} exhausted)")
                        print(f"  {ticker}: Rate limit saved - will skip this key for 24 hours")
                    
                    CURRENT_KEY_INDEX = (CURRENT_KEY_INDEX + 1) % len(ALPHAVANTAGE_KEYS)
                    
                    if len(RATE_LIMITED_KEYS) >= len(ALPHAVANTAGE_KEYS):
                        if debug:
                            print(f"  {ticker}: All keys exhausted!")
                        return [], "rate_limited_all"
                    
                    time.sleep(1)
                    continue
                
                return [], "api_error"
            
            # Parse earnings data
            if 'quarterlyEarnings' not in data:
                if debug:
                    print(f"  {ticker}: No 'quarterlyEarnings' field found")
                return [], "no_earnings"
            
            earnings_info = []
            for quarter in data['quarterlyEarnings']:
                reported_date = quarter.get('reportedDate')
                # API changed field name from 'reportedTime' to 'reportTime'
                reported_time = quarter.get('reportTime') or quarter.get('reportedTime', 'amc')
                
                if reported_date:
                    # Normalize time values: 'pre-market' -> 'bmo', 'post-market' -> 'amc'
                    time_map = {'pre-market': 'bmo', 'post-market': 'amc'}
                    normalized_time = time_map.get(reported_time.lower(), reported_time.lower())
                    
                    earnings_info.append({
                        'date': datetime.strptime(reported_date, '%Y-%m-%d'),
                        'time': normalized_time
                    })
            
            # Cache the results
            cache[ticker] = [
                {'date': e['date'].isoformat(), 'time': e['time']} 
                for e in earnings_info
            ]
            save_cache(cache)
            
            if debug:
                print(f"  {ticker}: ✓ Success with key ...{key_suffix}")
            
            return sorted(earnings_info, key=lambda x: x['date'], reverse=True), "success"
        
        except Exception as e:
            if debug:
                print(f"  {ticker}: Exception - {str(e)[:80]}")
            if attempt < max_attempts - 1:
                CURRENT_KEY_INDEX = (CURRENT_KEY_INDEX + 1) % len(ALPHAVANTAGE_KEYS)
                continue
            return [], "exception"
    
    return [], "unknown_error"

# ============================================================================
# YAHOO FINANCE - PRICE DATA
# ============================================================================

def get_yahoo_price_data(ticker, start_date, end_date):
    """Get historical closing prices from Yahoo Finance"""
    start_ts = int(start_date.timestamp())
    end_ts = int(end_date.timestamp())
    
    url = f"https://query1.finance.yahoo.com/v8/finance/chart/{ticker}"
    params = {'period1': start_ts, 'period2': end_ts, 'interval': '1d'}
    headers = {'User-Agent': 'Mozilla/5.0'}
    
    try:
        response = requests.get(url, params=params, headers=headers, timeout=10)
        data = response.json()
        
        result = data['chart']['result'][0]
        timestamps = result['timestamp']
        closes = result['indicators']['quote'][0]['close']
        
        df = pd.DataFrame({
            'date': [datetime.fromtimestamp(ts) for ts in timestamps],
            'close': closes
        })
        df.set_index('date', inplace=True)
        df.dropna(inplace=True)
        
        return df
    
    except Exception as e:
        print(f"❌ Error fetching prices for {ticker}: {e}")
        return pd.DataFrame()

# ============================================================================
# HELPER FUNCTIONS
# ============================================================================

def find_nearest_price(price_data, target_date):
    """Find closing price on nearest trading day"""
    if price_data.empty:
        return None, None
    
    start = target_date - timedelta(days=7)
    end = target_date + timedelta(days=7)
    nearby = price_data[(price_data.index >= start) & (price_data.index <= end)]
    
    if nearby.empty:
        return None, None
    
    time_diffs = (nearby.index - target_date).to_series().abs()
    closest_idx = time_diffs.argmin()
    return nearby.iloc[closest_idx]['close'], nearby.index[closest_idx]

def get_reference_price(price_data, earnings_date, timing):
    """Get entry price based on earnings timing"""
    target_date = earnings_date - timedelta(days=1) if timing == 'bmo' else earnings_date
    return find_nearest_price(price_data, target_date)

def calculate_historical_volatility(price_data, earnings_date, lookback_days=30):
    """Calculate annualized historical volatility"""
    end_date = earnings_date - timedelta(days=1)
    start_date = end_date - timedelta(days=lookback_days + 10)
    
    window = price_data[(price_data.index >= start_date) & (price_data.index <= end_date)]
    
    if len(window) < 20:
        return None
    
    returns = window['close'].pct_change().dropna()
    daily_vol = returns.std()
    annual_vol = daily_vol * np.sqrt(252)
    
    return annual_vol

def get_volatility_tier(hvol):
    """Map historical volatility to strike width multiplier"""
    hvol_pct = hvol * 100
    
    if hvol_pct < 25:
        return 1.0
    elif hvol_pct < 35:
        return 1.2
    elif hvol_pct < 45:
        return 1.4
    else:
        return 1.5

def calculate_stats(data):
    """
    Calculate containment and directional statistics
    
    CHANGED v2: Added break_bias (directional bias among breaks only)
                and magnitude-weighted metrics
    """
    total = len(data)
    moves = np.array([d['move'] for d in data])
    widths = np.array([d['width'] for d in data])
    
    # Containment
    stays_within = sum(1 for i, m in enumerate(moves) if abs(m) <= widths[i])
    breaks_up = sum(1 for i, m in enumerate(moves) if m > widths[i])
    breaks_down = sum(1 for i, m in enumerate(moves) if m < -widths[i])
    
    # EXISTING: Overall directional bias (% of all moves that went up)
    up_moves = sum(1 for m in moves if m > 0)
    overall_bias = (up_moves / total) * 100
    
    # NEW v2: Break directional bias (among breaks only, which direction?)
    total_breaks = breaks_up + breaks_down
    if total_breaks > 0:
        break_bias = (breaks_up / total_breaks) * 100
    else:
        break_bias = 50.0  # Neutral if no breaks
    
    # NEW v2: Magnitude-weighted average move
    avg_move = np.mean(moves)
    avg_move_vs_width = (avg_move / np.mean(widths)) * 100 if np.mean(widths) > 0 else 0
    
    return {
        'total': total,
        'containment': (stays_within / total) * 100,
        'breaks_up': breaks_up,
        'breaks_down': breaks_down,
        'overall_bias': overall_bias,      # % of all moves that were positive
        'break_bias': break_bias,          # NEW: % of breaks that were upward
        'avg_move_pct': avg_move,          # NEW: Average move magnitude
        'drift_vs_width': avg_move_vs_width,  # NEW: Drift relative to strike width
        'avg_width': np.mean(widths)
    }

# ============================================================================
# MAIN ANALYSIS
# ============================================================================

def analyze_earnings_movement(ticker, lookback_quarters=24, verbose=True, debug=False):
    """Analyze post-earnings movements with volatility-adjusted strikes"""
    
    if verbose:
        print(f"\n{'='*75}")
        print(f"📊 {ticker} - Post-Earnings Containment Analysis")
        print(f"{'='*75}")
    
    # Get earnings dates
    earnings_info, status = get_earnings_details(ticker, debug=debug)
    if not earnings_info:
        return None, status
    
    # Filter to past earnings
    today = datetime.now()
    past_earnings = [e for e in earnings_info if e['date'] < today][:lookback_quarters]
    
    if len(past_earnings) < 10:
        if verbose:
            print(f"⚠️  Insufficient data: only {len(past_earnings)} earnings periods")
        return None, "insufficient_quarters"
    
    # Get price data
    oldest = min([e['date'] for e in past_earnings]) - timedelta(days=120)
    price_data = get_yahoo_price_data(ticker, oldest, today)
    
    if price_data.empty:
        return None, "no_price_data"
    
    # Collect movement data
    data_45 = []
    data_90 = []
    hvol_list = []
    
    if verbose:
        print(f"\nAnalyzing {len(past_earnings)} earnings periods...")
    
    for earnings in past_earnings:
        # Calculate historical volatility
        hvol = calculate_historical_volatility(price_data, earnings['date'])
        if hvol is None:
            continue
        
        hvol_list.append(hvol * 100)
        strike_std = get_volatility_tier(hvol)
        
        # Get entry price
        ref_price, ref_date = get_reference_price(price_data, earnings['date'], earnings['time'])
        if ref_price is None:
            continue
        
        # Calculate strike widths
        dte_45_factor = np.sqrt(45 / 365)
        dte_90_factor = np.sqrt(90 / 365)
        strike_width_45 = hvol * dte_45_factor * strike_std * 100
        strike_width_90 = hvol * dte_90_factor * strike_std * 100
        
        # Test 45-day outcome
        target_45 = earnings['date'] + timedelta(days=45)
        if target_45 <= today:
            price_45, date_45 = find_nearest_price(price_data, target_45)
            if price_45 is not None:
                move_45 = (price_45 - ref_price) / ref_price * 100
                data_45.append({
                    'move': move_45,
                    'width': strike_width_45,
                    'hvol': hvol * 100,
                    'date': earnings['date'].strftime('%Y-%m-%d')
                })
        
        # Test 90-day outcome
        target_90 = earnings['date'] + timedelta(days=90)
        if target_90 <= today:
            price_90, date_90 = find_nearest_price(price_data, target_90)
            if price_90 is not None:
                move_90 = (price_90 - ref_price) / ref_price * 100
                data_90.append({
                    'move': move_90,
                    'width': strike_width_90,
                    'hvol': hvol * 100,
                    'date': earnings['date'].strftime('%Y-%m-%d')
                })
    
    if len(data_45) < 10 or len(data_90) < 10:
        if verbose:
            print(f"⚠️  Insufficient valid data")
        return None, "insufficient_valid_data"
    
    # Calculate statistics
    stats_45 = calculate_stats(data_45)
    stats_90 = calculate_stats(data_90)
    avg_hvol = np.mean(hvol_list)
    avg_tier = get_volatility_tier(avg_hvol / 100)
    
    # CHANGED v2.1: Simplified logic - Containment vs Directional Bias vs Skip
    rec_parts = []
    bias_reasons = []
    
    # DEBUG: Print stats before logic
    if verbose:
        print(f"\n  🔍 DEBUG - Recommendation Logic:")
        print(f"     90D containment: {stats_90['containment']:.2f}%")
        print(f"     90D overall_bias: {stats_90['overall_bias']:.2f}%")
        print(f"     90D breaks: {stats_90['breaks_up']} up, {stats_90['breaks_down']} down")
        print(f"     90D drift: {stats_90['avg_move_pct']:+.2f}%")
        print(f"     45D containment: {stats_45['containment']:.2f}%")
    
    # ========================================================================
    # CATEGORY 1: CONTAINMENT STRATEGIES (IC)
    # ========================================================================
    if stats_90['containment'] >= 69.5:
        break_ratio = max(stats_90['breaks_up'], stats_90['breaks_down']) / (
            min(stats_90['breaks_up'], stats_90['breaks_down']) + 1
        )
        
        if break_ratio < 2:  # Breaks are balanced
            rec_parts.append("IC90")
            if verbose:
                print(f"     ✓ IC90: Balanced breaks (ratio={break_ratio:.2f})")
        elif stats_90['break_bias'] >= 70:  # Most breaks are upward
            rec_parts.append("IC90⚠↑")
            if verbose:
                print(f"     ✓ IC90⚠↑: Watch upside ({stats_90['break_bias']:.0f}% breaks upward)")
        else:  # Most breaks are downward
            rec_parts.append("IC90⚠↓")
            if verbose:
                print(f"     ✓ IC90⚠↓: Watch downside ({100-stats_90['break_bias']:.0f}% breaks downward)")
            
    elif stats_45['containment'] >= 69.5:
        if verbose:
            print(f"     → 90D didn't qualify, checking 45D...")
        
        break_ratio = max(stats_45['breaks_up'], stats_45['breaks_down']) / (
            min(stats_45['breaks_up'], stats_45['breaks_down']) + 1
        )
        
        if break_ratio < 2:
            rec_parts.append("IC45")
            if verbose:
                print(f"     ✓ IC45: Balanced breaks (ratio={break_ratio:.2f})")
        elif stats_45['break_bias'] >= 70:
            rec_parts.append("IC45⚠↑")
            if verbose:
                print(f"     ✓ IC45⚠↑: Watch upside ({stats_45['break_bias']:.0f}% breaks upward)")
        else:
            rec_parts.append("IC45⚠↓")
            if verbose:
                print(f"     ✓ IC45⚠↓: Watch downside ({100-stats_45['break_bias']:.0f}% breaks downward)")
    
    # ========================================================================
    # CATEGORY 2: DIRECTIONAL BIAS (No containment edge, but directional signal)
    # ========================================================================
    else:
        # Check for UPWARD edge (any of these conditions)
        has_upward_edge = False
        
        if stats_90['overall_bias'] >= 65:
            has_upward_edge = True
            bias_reasons.append(f"{stats_90['overall_bias']:.0f}% bias")
        
        if stats_90['breaks_up'] >= stats_90['breaks_down'] * 1.5 and stats_90['breaks_up'] >= 2:
            has_upward_edge = True
            bias_reasons.append(f"{stats_90['breaks_up']}:{stats_90['breaks_down']}↑ breaks")
        
        if stats_90['avg_move_pct'] >= 3.0:
            has_upward_edge = True
            bias_reasons.append(f"{stats_90['avg_move_pct']:+.1f}% drift")
        
        # Check for DOWNWARD edge
        has_downward_edge = False
        
        if stats_90['overall_bias'] <= 35:
            has_downward_edge = True
            bias_reasons.append(f"{stats_90['overall_bias']:.0f}% bias")
        
        if stats_90['breaks_down'] >= stats_90['breaks_up'] * 1.5 and stats_90['breaks_down'] >= 2:
            has_downward_edge = True
            bias_reasons.append(f"{stats_90['breaks_up']}:{stats_90['breaks_down']}↓ breaks")
        
        if stats_90['avg_move_pct'] <= -3.0:
            has_downward_edge = True
            bias_reasons.append(f"{stats_90['avg_move_pct']:+.1f}% drift")
        
        # Assign directional recommendation
        if has_upward_edge:
            reason_str = ", ".join(bias_reasons)
            rec_parts.append(f"BIAS↑ ({reason_str})")
            if verbose:
                print(f"     ✓ Upward edge detected: {reason_str}")
        elif has_downward_edge:
            reason_str = ", ".join(bias_reasons)
            rec_parts.append(f"BIAS↓ ({reason_str})")
            if verbose:
                print(f"     ✓ Downward edge detected: {reason_str}")
    
    # ========================================================================
    # FINAL RECOMMENDATION
    # ========================================================================
    recommendation = rec_parts[0] if rec_parts else "SKIP"
    
    if verbose:
        print(f"     Final recommendation: {recommendation}")
    
    # Print results
    if verbose:
        print(f"\n📊 {ticker} | {avg_hvol:.1f}% HVol | {avg_tier:.1f} std (±{stats_90['avg_width']:.1f}%)")
        print(f"\n  45-Day: {stats_45['total']}/{lookback_quarters} tested")
        print(f"    Containment: {stats_45['containment']:.0f}%")
        print(f"    Breaks: Up {stats_45['breaks_up']}, Down {stats_45['breaks_down']}")
        print(f"    Overall Bias: {stats_45['overall_bias']:.0f}% up")
        print(f"    Break Bias: {stats_45['break_bias']:.0f}% of breaks were upward")
        print(f"    Avg Drift: {stats_45['avg_move_pct']:+.1f}% ({stats_45['drift_vs_width']:+.0f}% of width)")
        
        print(f"\n  90-Day: {stats_90['total']}/{lookback_quarters} tested")
        print(f"    Containment: {stats_90['containment']:.0f}%")
        print(f"    Breaks: Up {stats_90['breaks_up']}, Down {stats_90['breaks_down']}")
        print(f"    Overall Bias: {stats_90['overall_bias']:.0f}% up")
        print(f"    Break Bias: {stats_90['break_bias']:.0f}% of breaks were upward")
        print(f"    Avg Drift: {stats_90['avg_move_pct']:+.1f}% ({stats_90['drift_vs_width']:+.0f}% of width)")
        
        print(f"\n  💡 Strategy: {recommendation}")
    
    summary = {
        'ticker': ticker,
        'hvol': round(avg_hvol, 1),
        'tier': round(avg_tier, 1),
        'strike_width': round(stats_90['avg_width'], 1),
        '45d_contain': round(stats_45['containment'], 0),
        '45d_breaks_up': stats_45['breaks_up'],
        '45d_breaks_dn': stats_45['breaks_down'],
        '45d_overall_bias': round(stats_45['overall_bias'], 0),
        '45d_break_bias': round(stats_45['break_bias'], 0),
        '45d_drift': round(stats_45['avg_move_pct'], 1),
        '90d_contain': round(stats_90['containment'], 0),
        '90d_breaks_up': stats_90['breaks_up'],
        '90d_breaks_dn': stats_90['breaks_down'],
        '90d_overall_bias': round(stats_90['overall_bias'], 0),
        '90d_break_bias': round(stats_90['break_bias'], 0),
        '90d_drift': round(stats_90['avg_move_pct'], 1),
        'strategy': recommendation
    }
    
    return summary, "success"

# ============================================================================
# BATCH PROCESSING
# ============================================================================

def format_break_ratio(up_breaks, down_breaks, break_bias):
    """
    Format break ratio with directional arrow if 2:1 edge exists
    
    CHANGED v2: Now uses break_bias to determine arrow direction
    """
    if up_breaks == 0 and down_breaks == 0:
        return "0:0"
    
    # Use break_bias for more accurate directional assessment
    if break_bias >= 66.7:  # 2:1 or better upward
        return f"{up_breaks}:{down_breaks}↑"
    elif break_bias <= 33.3:  # 2:1 or better downward
        return f"{up_breaks}:{down_breaks}↓"
    else:
        return f"{up_breaks}:{down_breaks}"

def batch_analyze(tickers, lookback_quarters=24, debug=False, fetch_iv=True):
    """Analyze multiple tickers with progress tracking"""
    
    print("\n" + "="*75)
    print(f"EARNINGS CONTAINMENT ANALYZER - v2.2")
    print(f"Lookback: {lookback_quarters} quarters (~{lookback_quarters/4:.0f} years)")
    print(f"Volatility Tiers: <25%=1.0std | 25-35%=1.2std | 35-45%=1.4std | >45%=1.5std")
    print(f"Logic: Containment (IC) vs Directional Bias vs Skip")
    if fetch_iv and YFINANCE_AVAILABLE:
        print(f"NEW v2.2: Current IV from Yahoo Finance (15-20min delayed)")
    print("="*75)
    
    # NEW v2.1: Show rate limit status at start
    rate_limited_keys = load_rate_limits()
    if rate_limited_keys:
        available = len(ALPHAVANTAGE_KEYS) - len(rate_limited_keys)
        print(f"\n⚠️  Rate Limit Status: {available}/{len(ALPHAVANTAGE_KEYS)} API keys available")
        print(f"   {len(rate_limited_keys)} key(s) exhausted, will reset in ~24hrs")
    else:
        print(f"\n✓ All {len(ALPHAVANTAGE_KEYS)} API keys available")
    
    results = []
    fetch_summary = {'cached': [], 'api': [], 'failed': []}
    iv_summary = {'success': [], 'failed': []}
    
    for i, ticker in enumerate(tickers, 1):
        print(f"\r[{i}/{len(tickers)}] Processing {ticker}...", end='', flush=True)
        
        cache = load_cache()
        from_cache = ticker in cache
        
        summary, status = analyze_earnings_movement(ticker, lookback_quarters, verbose=False, debug=debug)
        
        if summary:
            # NEW v2.2: Fetch current IV
            if fetch_iv and YFINANCE_AVAILABLE:
                iv_data = get_current_iv(ticker, dte_target=45)
                if iv_data:
                    summary['current_iv'] = iv_data['iv']
                    summary['iv_dte'] = iv_data['dte']
                    # Calculate IV premium vs historical volatility
                    iv_premium = ((iv_data['iv'] - summary['hvol']) / summary['hvol']) * 100
                    summary['iv_premium'] = round(iv_premium, 1)
                    iv_summary['success'].append(ticker)
                else:
                    summary['current_iv'] = None
                    summary['iv_dte'] = None
                    summary['iv_premium'] = None
                    iv_summary['failed'].append(ticker)
            else:
                summary['current_iv'] = None
                summary['iv_dte'] = None
                summary['iv_premium'] = None
            
            results.append(summary)
            if from_cache:
                fetch_summary['cached'].append(ticker)
            else:
                fetch_summary['api'].append(ticker)
        else:
            fetch_summary['failed'].append(ticker)
        
        time.sleep(0.5)
    
    print("\r" + " " * 80 + "\r", end='')
    
    # Print fetch summary
    print(f"\n📊 FETCH SUMMARY")
    print(f"{'='*75}")
    if fetch_summary['cached']:
        print(f"✓ Earnings From Cache ({len(fetch_summary['cached'])}): {', '.join(fetch_summary['cached'][:5])}{'...' if len(fetch_summary['cached']) > 5 else ''}")
    if fetch_summary['api']:
        print(f"✓ Earnings From API ({len(fetch_summary['api'])}): {', '.join(fetch_summary['api'])}")
    if fetch_iv and YFINANCE_AVAILABLE:
        if iv_summary['success']:
            print(f"✓ IV Data Retrieved ({len(iv_summary['success'])}): {', '.join(iv_summary['success'][:5])}{'...' if len(iv_summary['success']) > 5 else ''}")
        if iv_summary['failed']:
            print(f"✗ IV Data Failed ({len(iv_summary['failed'])}): {', '.join(iv_summary['failed'])}")
    if fetch_summary['failed']:
        print(f"✗ Analysis Failed ({len(fetch_summary['failed'])}): {', '.join(fetch_summary['failed'])}")
    
    if not results:
        print("\n⚠️  No valid results")
        return None
    
    df = pd.DataFrame(results)
    
    # Calculate 45-day strike width
    df['45d_width'] = df.apply(lambda x: round(x['strike_width'] * np.sqrt(45/90), 1), axis=1)
    
    # Format break ratios using new break_bias
    df['45_break_fmt'] = df.apply(
        lambda x: format_break_ratio(x['45d_breaks_up'], x['45d_breaks_dn'], x['45d_break_bias']), 
        axis=1
    )
    df['90_break_fmt'] = df.apply(
        lambda x: format_break_ratio(x['90d_breaks_up'], x['90d_breaks_dn'], x['90d_break_bias']), 
        axis=1
    )
    
    # Format strategies - now includes reason strings
    df['strategy_fmt'] = df['strategy'].apply(lambda x: x if len(x) <= 20 else x.split('(')[0].strip())
    
    # For display, shorten BIAS reasons
    df['strategy_display'] = df['strategy'].apply(lambda x: 
        x.replace('BIAS↑', 'BIAS↑').replace('BIAS↓', 'BIAS↓') if 'BIAS' in x else x
    )
    
    # Display results
    print(f"\n{'='*130}")
    print("EARNINGS BACKTEST RESULTS (v2.2 - with Current IV Context)")
    print("="*130)
    print("Historical: HVol = past volatility | Contain% = stayed within strikes | Bias = % up moves | Drift = avg move")
    print("Current: CurIV = current implied vol | IVPrem = IV vs historical (+ means elevated)")
    print("Note: This shows what happened historically. Current IV context helps assess if premium is attractive NOW.")
    print("="*130)
    
    # Prepare display columns
    display_cols = {
        'Ticker': df['ticker'],
        'HVol%': df['hvol'].astype(int),
    }
    
    # Add IV columns if available
    if 'current_iv' in df.columns and df['current_iv'].notna().any():
        display_cols['CurIV%'] = df['current_iv'].apply(lambda x: f"{int(x)}" if pd.notna(x) else "N/A")
        display_cols['IVPrem'] = df['iv_premium'].apply(lambda x: f"{x:+.0f}%" if pd.notna(x) else "N/A")
        display_cols['|'] = '|'
    
    display_cols.update({
        '90D%': df['90d_contain'].astype(int),
        '90Bias': df['90d_overall_bias'].astype(int),
        '90Break': df['90_break_fmt'],
        '90Drift': df['90d_drift'].apply(lambda x: f"{x:+.1f}%"),
        ' | ': '|',
        'Pattern': df['strategy_display']
    })
    
    display_df = pd.DataFrame(display_cols)
    
    print(display_df.to_string(index=False))
    
    # Summary insights
    print(f"\n{'='*130}")
    print("KEY TAKEAWAYS:")
    print("="*130)
    
    # Simplified pattern counts
    ic_count = len(df[df['strategy_fmt'].str.contains('IC', na=False)])
    bias_up_count = len(df[df['strategy_fmt'].str.contains('BIAS↑', na=False)])
    bias_down_count = len(df[df['strategy_fmt'].str.contains('BIAS↓', na=False)])
    skip_count = len(df[df['strategy_fmt'] == 'SKIP'])
    
    print(f"\n📊 Pattern Summary: {ic_count} IC candidates | {bias_up_count} Upward bias | {bias_down_count} Downward bias | {skip_count} No edge")
    
    # IV Context (if available) - CONDENSED
    if 'iv_premium' in df.columns and df['iv_premium'].notna().any():
        elevated = df[df['iv_premium'] >= 15].sort_values('iv_premium', ascending=False)
        depressed = df[df['iv_premium'] <= -15].sort_values('iv_premium')
        
        print(f"\n💰 IV Landscape:")
        if not elevated.empty:
            tickers_str = ', '.join([f"{row['ticker']}(+{row['iv_premium']:.0f}%)" for _, row in elevated.head(5).iterrows()])
            print(f"  Rich Premium (IV elevated ≥15%): {tickers_str}")
        if not depressed.empty:
            tickers_str = ', '.join([f"{row['ticker']}({row['iv_premium']:.0f}%)" for _, row in depressed.head(3).iterrows()])
            print(f"  Thin Premium (IV depressed ≤-15%): {tickers_str}")
        
        normal_count = len(df[(df['iv_premium'] > -15) & (df['iv_premium'] < 15)])
        print(f"  Normal Range: {normal_count} tickers")
    
    # Asymmetric ICs - CONDENSED
    ic_up_skew = df[(df['strategy_fmt'].str.contains('IC.*⚠↑', regex=True, na=False))]
    ic_down_skew = df[(df['strategy_fmt'].str.contains('IC.*⚠↓', regex=True, na=False))]
    
    if not ic_up_skew.empty or not ic_down_skew.empty:
        print(f"\n⚠️  Asymmetric ICs:")
        if not ic_up_skew.empty:
            print(f"  Upside risk: {', '.join(ic_up_skew['ticker'].tolist())}")
        if not ic_down_skew.empty:
            print(f"  Downside risk: {', '.join(ic_down_skew['ticker'].tolist())}")
    
    # Strong directional edges - CONDENSED
    strong_bias = df[
        (df['strategy_fmt'].str.contains('BIAS', na=False)) &
        ((df['90d_overall_bias'] >= 70) | (df['90d_overall_bias'] <= 30))
    ]
    if not strong_bias.empty:
        print(f"\n📈 Strong Directional Signals:")
        for _, row in strong_bias.iterrows():
            direction = "↑" if row['90d_overall_bias'] >= 70 else "↓"
            print(f"  {row['ticker']}: {row['90d_overall_bias']:.0f}% bias {direction}, {row['90_break_fmt']} breaks, {row['90d_drift']:+.1f}% drift")
    
    print(f"\n💡 Remember: Past patterns ≠ Future results. IV context shows current opportunity cost.")
    
    return df

# ============================================================================
# RUN
# ============================================================================

if __name__ == "__main__":
    # Test tickers
    tickers = ["DAL", "PEP", "FAST", "BLK", "C", "DPZ", "GS", "JNJ", "JPM", "WFC", 
               "OMC", "ABT", "BAC", "CFG", "MS", "PGR", "PLD", "PNC", "SYF", "JBHT", 
               "UAL", "BK", "KEY", "MMC", "MTB", "SCHW", "SNA", "TRV", "USB", "CSX", 
               "AXP", "FITB", "HBAN", "RF", "SLB", "STT", "TFC", "STLD"]
    
    results = batch_analyze(tickers, lookback_quarters=24, debug=False)


EARNINGS CONTAINMENT ANALYZER - v2.2
Lookback: 24 quarters (~6 years)
Volatility Tiers: <25%=1.0std | 25-35%=1.2std | 35-45%=1.4std | >45%=1.5std
Logic: Containment (IC) vs Directional Bias vs Skip
NEW v2.2: Current IV from Yahoo Finance (15-20min delayed)

✓ All 4 API keys available
                                                                                
📊 FETCH SUMMARY
✓ Earnings From Cache (38): DAL, PEP, FAST, BLK, C...
✓ IV Data Retrieved (38): DAL, PEP, FAST, BLK, C...

EARNINGS BACKTEST RESULTS (v2.2 - with Current IV Context)
Historical: HVol = past volatility | Contain% = stayed within strikes | Bias = % up moves | Drift = avg move
Current: CurIV = current implied vol | IVPrem = IV vs historical (+ means elevated)
Note: This shows what happened historically. Current IV context helps assess if premium is attractive NOW.
Ticker  HVol% CurIV% IVPrem |  90D%  90Bias 90Break 90Drift  |                                     Pattern
   DAL     43     49   +14% |    67      62 